In [2]:
import hsfs
import hopsworks
import configparser

def retrieve_final_df_feature_group():
    try:
        # Read the API key from config.ini
        config = configparser.ConfigParser()
        config.read('/Users/pavankumarradhala/Desktop/projects/Truck_delay/src/config/config.ini')

        # Extract the API key
        api_key = config['API']['hopswork_api_key']

        # Connect to the Hopsworks feature store using only the API key
        connection = hopsworks.connection(
            api_key_value=api_key
        )

        # Get the project and feature store
        project = connection.get_project()
        fs = project.get_feature_store()

        # Retrieve the 'final_df_feature_group' feature group
        final_df_fg = fs.get_feature_group(name="final_df_feature_group", version=1)

        # Retrieve the feature group data as a Pandas DataFrame
        final_df = final_df_fg.read()

        # Close the connection
        connection.close()

        # Return the final DataFrame
        return final_df

    except Exception as e:
        print(f"Error retrieving feature group: {e}")
        return None

# Usage
final_df = retrieve_final_df_feature_group()
if final_df is not None:
    print(final_df.head())  # Display the first few rows


Error retrieving feature group: 'API'
